### **Aplicação da FFT e Wavelet para remover ruído de sons**

---

Gabriel Oukawa <br>
Álgebra linear para ciência de dados <br>
2º Semestre de 2025

---


# Exemplo 1: Conversação

Bibliotecas

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import requests
import io
import pywt
from IPython.display import Audio

Carregar dados de entrada (som utilizado: conversação)

In [40]:
url = 'https://keithito.com/LJ-Speech-Dataset/LJ025-0076.wav'
response = requests.get(url)
rate, B = wavfile.read(io.BytesIO(response.content))

# Converter de stereo para mono
if B.ndim > 1:
    B = B.mean(axis=1)

B = B.astype(float)

print('Audio original')
display(Audio(B, rate=rate, autoplay=False))

Audio original


Adicionar ruído (white noise) e aplicar FFT para tentar remover (passa baixa)

In [41]:
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams.update({'font.size': 8})

if B.ndim > 1:
    B = B.mean(axis=1)

B = B.astype(float)

noise_scale = 0.001 * 32768
Bnoise = B + np.random.normal(0, noise_scale, B.shape)

Bt = np.fft.fft(Bnoise)
Btshift = np.fft.fftshift(Bt)
F = np.log(np.abs(Btshift)+1)

n = len(B)
freqs = np.fft.fftshift(np.fft.fftfreq(n, 1/rate))

ind = np.abs(freqs) < 4000
Btshiftfilt = Btshift * ind
Ffilt = np.log(np.abs(Btshiftfilt)+1)

Btfilt = np.fft.ifftshift(Btshiftfilt)
Bfilt = np.fft.ifft(Btfilt).real

fig, axs = plt.subplots(2, 3, figsize=(18, 8))

signals = [B, Bnoise, Bfilt]
titles = ['Original', 'Com Ruído', 'Filtrado']

for j in range(3):
    axs[0, j].plot(signals[j], color='k', linewidth=0.25)
    axs[0, j].set_title(titles[j])
    axs[0, j].axis('tight')

for j in range(3):
    axs[1, j].specgram(signals[j], Fs=rate, NFFT=1024, noverlap=512, cmap='jet')
    axs[1, j].set_title(f'{titles[j]} - Espectrograma')
    axs[1, j].set_xlabel('Tempo [s]')
    axs[1, j].set_ylabel('Frequência [Hz]')

plt.tight_layout()
plt.show()

print('Ruído puro adicionado')
display(Audio(Bnoise - B, rate=rate, autoplay=False))
print('Audio com ruído')
display(Audio(Bnoise, rate=rate, autoplay=False))
print('Audio filtrado')
display(Audio(Bfilt, rate=rate, autoplay=False))

Output hidden; open in https://colab.research.google.com to view.

Adicionar ruído (white noise) e aplicar Wavelet para tentar remover

In [42]:
if B.ndim > 1:
    B = B.mean(axis=1)

B = B.astype(float)

noise_scale = 0.001 * 32768
Bnoise = B + np.random.normal(0, noise_scale, B.shape)

n = 2
w = 'haar'

coeffs = pywt.wavedec(Bnoise, wavelet=w, level=n)
sigma = np.median(np.abs(coeffs[-1])) / 0.6745
thresh = sigma * np.sqrt(2*np.log(len(B)))

coeffs_filt = [coeffs[0]]
for c in coeffs[1:]:
    coeffs_filt.append(pywt.threshold(c, thresh, mode='soft'))

Bfilt = pywt.waverec(coeffs_filt, wavelet=w)

fig, axs = plt.subplots(2, 3, figsize=(18, 8))

signals = [B, Bnoise, Bfilt]
titles = ['Original', 'Com Ruído', 'Filtrado']

for j in range(3):
    axs[0, j].plot(signals[j], color='k', linewidth=0.25)
    axs[0, j].set_title(titles[j])
    axs[0, j].axis('tight')

for j in range(3):
    axs[1, j].specgram(signals[j], Fs=rate, NFFT=1024, noverlap=512, cmap='jet')
    axs[1, j].set_title(f'{titles[j]} - Espectrograma')
    axs[1, j].set_xlabel('Tempo [s]')
    axs[1, j].set_ylabel('Frequência [Hz]')

plt.tight_layout()
plt.show()

print('Ruído puro adicionado')
display(Audio(Bnoise - B, rate=rate, autoplay=False))
print('Audio com ruído')
display(Audio(Bnoise, rate=rate, autoplay=False))
print('Audio filtrado')
display(Audio(Bfilt, rate=rate, autoplay=False))

Output hidden; open in https://colab.research.google.com to view.

# Exemplo 2: Música

Bibliotecas

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import requests
import io
import pywt
import gdown
import librosa
from IPython.display import Audio

Carregar dados de entrada (som utilizado: música)

In [50]:
gdown.download(id='13gCP93BGr7LPVONWKRW0PFSM68QjZOna', quiet=True)

B, rate = librosa.load('cello_phrase_mezzo.mp3', sr=None, mono=True)

print("Audio original")
display(Audio(B, rate=rate, autoplay=False))

Output hidden; open in https://colab.research.google.com to view.

Adicionar ruído (white noise) e aplicar FFT para tentar remover

In [51]:
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams.update({'font.size': 8})

B = B.astype(float)

noise_scale = 0.001
Bnoise = B + np.random.normal(0, noise_scale, B.shape)

Bt = np.fft.fft(Bnoise)
Btshift = np.fft.fftshift(Bt)
F = np.log(np.abs(Btshift)+1)

n = len(B)
freqs = np.fft.fftshift(np.fft.fftfreq(n, 1/rate))

ind = np.abs(freqs) < 4000
Btshiftfilt = Btshift * ind
Ffilt = np.log(np.abs(Btshiftfilt)+1)

Btfilt = np.fft.ifftshift(Btshiftfilt)
Bfilt = np.fft.ifft(Btfilt).real

fig, axs = plt.subplots(2, 3, figsize=(18, 8))

signals = [B, Bnoise, Bfilt]
titles = ['Original', 'Com Ruído', 'Filtrado']

for j in range(3):
    axs[0, j].plot(signals[j], color='k', linewidth=0.25)
    axs[0, j].set_title(titles[j])
    axs[0, j].axis('tight')

for j in range(3):
    axs[1, j].specgram(signals[j], Fs=rate, NFFT=1024, noverlap=512, cmap='jet')
    axs[1, j].set_title(f'{titles[j]} - Espectrograma')
    axs[1, j].set_xlabel('Tempo [s]')
    axs[1, j].set_ylabel('Frequência [Hz]')

plt.tight_layout()
plt.show()

print('Ruído puro adicionado')
display(Audio(Bnoise - B, rate=rate, autoplay=False))
print('Audio com ruído')
display(Audio(Bnoise, rate=rate, autoplay=False))
print('Audio filtrado')
display(Audio(Bfilt, rate=rate, autoplay=False))

Output hidden; open in https://colab.research.google.com to view.

Adicionar ruído (white noise) e aplicar Wavelet para tentar remover

In [52]:
B = B.astype(float)

noise_scale = 0.001
Bnoise = B + np.random.normal(0, noise_scale, B.shape)

n = 2
w = 'haar'

coeffs = pywt.wavedec(Bnoise, wavelet=w, level=n)
sigma = np.median(np.abs(coeffs[-1])) / 0.6745
thresh = sigma * np.sqrt(2*np.log(len(B)))

coeffs_filt = [coeffs[0]]
for c in coeffs[1:]:
    coeffs_filt.append(pywt.threshold(c, thresh, mode='soft'))

Bfilt = pywt.waverec(coeffs_filt, wavelet=w)

fig, axs = plt.subplots(2, 3, figsize=(18, 8))

signals = [B, Bnoise, Bfilt]
titles = ['Original', 'Com Ruído', 'Filtrado']

for j in range(3):
    axs[0, j].plot(signals[j], color='k', linewidth=0.25)
    axs[0, j].set_title(titles[j])
    axs[0, j].axis('tight')

for j in range(3):
    axs[1, j].specgram(signals[j], Fs=rate, NFFT=1024, noverlap=512, cmap='jet')
    axs[1, j].set_title(f'{titles[j]} - Espectrograma')
    axs[1, j].set_xlabel('Tempo [s]')
    axs[1, j].set_ylabel('Frequência [Hz]')

plt.tight_layout()
plt.show()

print('Ruído puro adicionado')
display(Audio(Bnoise - B, rate=rate, autoplay=False))
print('Audio com ruído')
display(Audio(Bnoise, rate=rate, autoplay=False))
print('Audio filtrado')
display(Audio(Bfilt, rate=rate, autoplay=False))

Output hidden; open in https://colab.research.google.com to view.